### Syllabification Notebook
- This notebook takes WAV datasets generated by `1.0-segment-song-from-wavs` and segments the WAVs into syllables 
  - WAVs are expected to be in this format: `2017-04-16_17-27-44-760000.wav`
- The notebook outputs a textgrid file corresponding to each wav

### Import Packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import numpy as np
from datetime import datetime
import pandas as pd
import copy
from glob import glob

from avgn.utils.paths import DATA_DIR, ensure_dir
from avgn.utils.general import save_dict_pickle
from avgn.signalprocessing.spectrogramming import _build_mel_basis

### Parameters for segmenting syllables

In [4]:
# the size of the syllables (pixels*pixels)
syll_size = 128

# parameters for filtering
filtering_params = {
    # filtering
    "highcut": 15000,
    "lowcut": 500,
}

spectrogramming_params = {
    # spectrograms
    "mel_filter": False,  # should a mel filter be used?
    "num_freq": 512,  # how many channels to use in a spectrogram
    "sample_rate": 44100,  # what rate are your WAVs sampled at?
    "preemphasis": 0.97,
    "min_silence_for_spec": 0.5,  # minimum length of silence for a spectrogram to be considered a good spectrogram
    "max_vocal_for_spec": 5.0,  # the longest a single vocalization (protosyllable) is allowed to be
    "frame_shift_ms": 0.5,  # step size for fft
    "frame_length_ms": 6,  # frame length for fft
    "min_level_dB": -70,  # minimum threshold db for computing spe
    "min_level_dB_floor": -20,  # (db)
    "spec_thresh_delta_dB": 5,  # (db) what
    "ref_level_dB": 20,  # reference db for computing spec
    "sample_rate": 44100,  # sample rate of your data
}

envelope_params = {
    # Vocal Envelope
    "smoothing": "gaussian",  # 'none',
    "envelope_signal": "spectrogram",  # spectrogram or waveform, what to get the vocal envelope from
    "gauss_sigma_s": 0.0001,
    "FOI_min": 4,  # minimum frequency of interest for vocal envelope (in terms of mel)
    "FOI_max": 24,  # maximum frequency of interest for vocal envelope (in terms of mel)
}

bout_threshold_params = {
    # Silence Thresholding
    "silence_threshold": 0.001,  # normalized threshold for silence
    "min_len": 5.0,  # minimum length for a vocalization (fft frames)
    "power_thresh": 0.3,  # Threshold for which a syllable is considered to be quiet weak and is probably noise
}

syllabification_params = {
    # Syllabification
    "min_syll_len_s": 0.03,  # minimum length for a syllable
    "segmentation_rate": 0.0,  # 0.125, # rate at which to dynamically raise the segmentation threshold (ensure short syllables)
    "min_num_sylls": 20,  # min number of syllables to be considered a bout
}

hparams = {"species": "BF", "dataset": "Koumura_Okanoya"}
for d in [
    filtering_params,
    spectrogramming_params,
    envelope_params,
    bout_threshold_params,
    syllabification_params,
]:
    for k, v in d.items():
        hparams[k] = v

In [ ]:
# this is used to identify this training instance
now_string = datetime.now().strftime(
    "%Y-%m-%d_%H-%M-%S"
)  
# save the dictionary so that we can reload it for recovering waveforms
dict_save = DATA_DIR / ("parameter_dictionaries/" + now_string + "_dict.pickle")
ensure_dir(dict_save)
save_dict_pickle(hparams, dict_save)
print(dict_save)

/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/parameter_dictionaries/2019-04-26_09-37-24_dict.pickle


## Segment bouts

#### Run through in parallel

In [ ]:
# find the data bird folders
dataset_location = DATA_DIR / hparams['species'] / hparams['dataset'] / 'bouts'
indv_folders = list(dataset_location.glob('*'))
indv_folders[:3]

[PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/BF/Koumura_Okanoya/bouts/Bird9'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/BF/Koumura_Okanoya/bouts/Bird3'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/BF/Koumura_Okanoya/bouts/Bird4')]

In [ ]:
# skip creating datasets that already exist
skip_existing = False 

# run through WAVs in parallel
parallel = True 
verbosity = 10 # how verbose parallel should be
n_jobs = 10 # how many jobs to run in parallel

# visualize the output of the algorithm for optimizing parameters
visualize = False 

# whether to save the dataset
save_dataset=True 

# whether or not to output text for debugging
verbose = False

# visualization
nex = 10 # how many example wavs to plot

In [ ]:
from tqdm.autonotebook import tqdm
import os
from sklearn.externals.joblib import Parallel, delayed
from avgn.bout_segmentation.dynamic_threshold_segmentation import textgrid_from_wav

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# loop through and make individual datasets
for indv_folder in tqdm(indv_folders):

    # get the birds name
    indv_name = indv_folder.name
    print(indv_name)

    # check if the file already exists
    textgrid_loc = indv_folder / "TextGrids"
    ensure_dir(textgrid_loc)

    # get wav_list
    wav_list = list((indv_folder / "wavs").glob("*.wav"))

    # if visualizing, make sure only to show a few elements
    if visualize == True:
        wav_list = wav_list[:nex]

    # loop through each bout wav extracting syllable data
    if parallel:
        with Parallel(n_jobs=n_jobs, verbose=verbosity) as parallel:
            syllable_data = parallel(
                delayed(textgrid_from_wav)(
                    wav_file,
                    textgrid_loc / (wav_file.stem + ".TextGrid"),
                    hparams,
                    visualize=visualize,
                    verbose=verbose,
                )
                for wav_file in tqdm(wav_list)
            )

    else:
        syllable_data = [
            textgrid_from_wav(
                wav_file,
                textgrid_loc / (wav_file.stem + ".TextGrid"),
                hparams,
                visualize=visualize,
                verbose=verbose,
            )
            for wav_file in tqdm(wav_list)
        ]

Bird9


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:   15.1s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   22.4s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   33.3s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   47.9s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:  1.0min
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:  1.3min
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:  1.6min
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  1.8min
[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed:  2.2min
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  2.5min
[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed:  2.9min
[Parallel(n_jobs=10)]: Done 151 out of 154 | elapsed:  3.4min remaining:    4.0s


Bird3


[Parallel(n_jobs=10)]: Done 154 out of 154 | elapsed:  3.4min finished


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    9.5s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   14.3s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   22.3s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   31.9s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:   42.4s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   53.2s
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  1.3min
[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed:  1.6min
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.9min
[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed:  2.2min
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:  2.5min
[Parallel(n_jobs=10)]: Done 161 tasks      | elapsed:  2.8min
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  3.1min
[Parallel(n_jobs=10)]: Done 201 tasks      | elapsed:  

...

[Parallel(n_jobs=10)]: Done 107 out of 107 | elapsed:  2.1min finished


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    7.5s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   15.4s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   19.8s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   26.0s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:   36.1s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   43.6s
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:   56.4s
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed:  1.4min
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.6min
[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed:  1.8min
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:  2.1min
[Parallel(n_jobs=10)]: Done 161 tasks      | elapsed:  2.4min
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  2.7min
[Parallel(n_jobs=10)]: Done 201 tasks      | elapsed:  

Bird2


[Parallel(n_jobs=10)]: Done 277 out of 277 | elapsed:  4.0min finished


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:   11.7s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   13.7s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   26.4s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   33.9s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:   46.0s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   53.7s
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  1.4min
[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed:  1.6min
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.9min
[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed:  2.2min
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:  2.5min
